In [129]:
%pylab

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [130]:
wavelength = []
intensity =[]
with open("slit9_of_Baldwin_et_al_1991_corrpos_sum_nall.dat","r") as f:
    for line in f.readlines():
        line = line.split(' ')
        wavelength.append(float(line[0]))
        intensity.append(float(line[1]))
        
wavelength = np.array(wavelength)
intensity = np.array(intensity)

In [131]:
plot(wavelength,intensity)

In [132]:
import pyfits
image = np.copy(pyfits.open("emhalpha_flux.fits")[0].data)

In [147]:
imshow(image)

In [134]:
plot(intensity)

In [7]:
def thresh(y,y0):
    """
    Find all intervals above a threshold
    """
    intervals = []
    idx = []
    
    a = None
    high = False
    
    for i,yy in enumerate(y):
        
        if yy >= y0:            
            if high == False:
                a = i
                high = True
        else:
            if high == True:
                intervals.append(y[a:i])
                idx.append(a)
                a = None
            high = False
            
    return np.array(idx), np.array(intervals)
             
        


In [149]:
def ndflux(image,bins=None):
    
    if bins == None:
        bins = .001 * np.size(image)

    h,b = np.histogram(image,bins=bins)
    b = b[:-1] + .5*np.diff(b)
    dflux = b * h    
    ndflux = dflux/np.sum(dflux)
    
    return ndflux

def perimeter(image, a, w,ndf=None):
    
    if bins == None:
        bins = .001 * np.size(image)
        
    if ndf == None:
        ndf = ndflux(image)
    
    perimeter_bins = b[np.logical_and( (np.cumsum(ndf) > a-w) , ((np.cumsum(ndf) < a+w)) )]
    low = perimeter_bins[0]
    high = perimeter_bins[-1]

    perimeter_x = []
    perimeter_y = []

    for i,row in enumerate(image):
        for j,val in enumerate(row):
            if low <= val <= high:
                perimeter_x.append(i)
                perimeter_y.append(j)
            
    return np.array(perimeter_x), np.array(perimeter_y)

def center(image):
    
    x,y = image.shape
    
    m,cx,cy = 0.,0.,0.
    
    for xx in range(x):
        for yy in range(y):
            val = image[xx][yy]
            cx += val*xx
            cy += val*yy
            m += val
        
    return cx/m, cy/m
        
def peak(image):
    i = np.argmax(image.flatten())
    lx,ly = image.shape
    return i/(ly),i%(ly)

def r_eff(px,py,cx,cy):
    
    r = np.sqrt((px-cx)**2 + (py-cy)**2)/(len(px))
    return np.sum(r)
    

In [148]:
import scipy.ndimage.filters

image2 = scipy.ndimage.filters.gaussian_filter(image,[50,50])

x,y=perimeter(image2,.1,.01,2000)

disp = np.copy(image2)

for xx,yy in zip(x,y):
    disp[xx][yy] = 0

cx,cy = center(image2)
cx2,cy2 = peak(image2)

imshow(disp)



In [46]:
r_eff(x,y,cx,cy)

765.23765838161523

In [108]:
r_eff(x,y,cx2,cy2)

792.96040232888652

In [145]:
a = np.linspace(.01,.3,25)
r = []
r2 = []
r3 = []
r4 = []
for aa in a:
    x,y=perimeter(image2,aa,.01,2000)
    r.append(r_eff(x,y,cx,cy))
    r2.append(r_eff(x,y,cx2,cy2))
    x,y=perimeter(image2,aa,.05,2000)
    r3.append(r_eff(x,y,cx,cy))
    x,y=perimeter(image,aa,.05,2000)
    r4.append(r_eff(x,y,cx,cy))
    

In [144]:
scatter(a,r)
scatter(a,r2)
scatter(a,r3)
scatter(a,r4)

In [153]:
plot(np.cumsum(np.sort(image.flatten()))/np.sum(image.flatten()))

In [159]:
plot(np.cumsum(ndflux(image,bins=image.size)))
plot(np.cumsum(ndflux(image,bins=2000)))